In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import StackingRegressor
from sklearn.linear_model import RidgeCV
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

# 读取数据
data = pd.read_csv('merged_results.csv')
# data.drop(columns=['Filename'])
# for col in data.columns:  
#     if data[col].dtype == 'object':  
#         le = LabelEncoder()  
#         data[col] = le.fit_transform(data[col])

data = pd.get_dummies(data)

# 选择高相关性特征
selected_features = [
    'O_n2', 'ForLoop', 'UnaryOperator', 'ContinueStmt', 'O_n3', 'O_1',
       'ArrayDecl', 'S_1', 'O_logn', 'TernaryOp', 'S_n2', 'closure', 'O_n',
       'FieldDecl', 'StringLiteral'
]

# 定义特征和目标变量
# X = data.drop(columns=['Time'])
X = data[selected_features]
y = data['Time']

# 标准化特征值
scaler = StandardScaler()
X = scaler.fit_transform(X)

# 分割数据为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定义基学习器
base_learners = [
    ('xgb', XGBRegressor(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)),
    ('rf', RandomForestRegressor(n_estimators=100, random_state=42)),
    ('svr', SVR(kernel='rbf', C=1, gamma=0.1))
]

# 定义堆叠回归模型
stacking_model = StackingRegressor(estimators=base_learners, final_estimator=RidgeCV())

# 训练模型
stacking_model.fit(X_train, y_train)

# 预测和评估模型
y_pred = stacking_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R-squared: {r2}')


Mean Squared Error: 6.317196965652587e-09
R-squared: -0.0006961086433021624
